# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

In [4]:
import findspark, pyspark
findspark.find()

'C:\\Bigdata\\spark-2.4.5-bin-hadoop2.7'

In [5]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession 
spark = SparkSession.builder.master('local').appName('tree').getOrCreate()

In [6]:
data = spark.read.csv('dog_food.csv', inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [9]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [18]:
data.show(5)

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
+---+---+----+---+-------+
only showing top 5 rows



In [24]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



In [25]:
data.groupBy('Spoiled').count().show()

+-------+-----+
|Spoiled|count|
+-------+-----+
|    0.0|  350|
|    1.0|  140|
+-------+-----+



In [26]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

In [28]:
indexer = StringIndexer(inputCol='Spoiled', outputCol='filter')

In [29]:
indexed = indexer.fit(data).transform(data)

In [30]:
indexed.head(5)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0, filter=1.0),
 Row(A=5, B=6, C=12.0, D=7, Spoiled=1.0, filter=1.0),
 Row(A=6, B=2, C=13.0, D=6, Spoiled=1.0, filter=1.0),
 Row(A=4, B=2, C=12.0, D=1, Spoiled=1.0, filter=1.0),
 Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0, filter=1.0)]

In [31]:
indexed.columns

['A', 'B', 'C', 'D', 'Spoiled', 'filter']

In [36]:
assembler = VectorAssembler(inputCols=[
      'A'
    , 'B'
    , 'C'
    , 'D'
    , 'Spoiled'
], outputCol='features')

In [37]:
output = assembler.transform(indexed)

In [40]:
output.select('features', 'A').show(5)

+--------------------+---+
|            features|  A|
+--------------------+---+
|[4.0,2.0,12.0,3.0...|  4|
|[5.0,6.0,12.0,7.0...|  5|
|[6.0,2.0,13.0,6.0...|  6|
|[4.0,2.0,12.0,1.0...|  4|
|[4.0,2.0,12.0,3.0...|  4|
+--------------------+---+
only showing top 5 rows



In [41]:
final_data = output.select('features', 'A')

In [42]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])